In [1]:
import loadList
import pandas as pd
import re, random

In [3]:
wordList_orig=loadList.main()

alps = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
freq = [8, 1, 2, 4, 13, 2, 2, 6, 6, 0, 0, 4, 2, 6, 7, 1, 0, 5, 6, 9, 2, 0, 2, 0, 2, 0]

freqs = pd.DataFrame({'alps': alps, 'freq': freq})
df = freqs
freqs = freqs.set_index('alps')
df['freq'] = df.freq.astype('int')
df = df.sort_values(by=['freq'], ascending=False)
df = df[df['freq']>6]
prefList = ['E','T','A','O']
df.head(10)
print(len(wordList_orig))

2315


In [4]:
setSize=5
def guess(wl):
    ls = getFilt(wl)
    hiFreq = ['a','t','o','i']
    for i in hiFreq: ls = list(filter(lambda x:x.find(i)>=0, ls))
    return ls

def eliminate(g, inWord, pred):
    ls = g
    if inWord in ls: ls.pop(ls.index(inWord))
    inarr   = list(zip([i for i in pred],[i for i in inWord]))
    remove  = list(filter(lambda x: x[0]=='0', inarr))
    
    for i in remove: ls = list(filter(lambda x:x.find(i[1])<0, ls))

    if '2' in pred:
        tmp = pred.replace('0','.')
        tmp = tmp.replace('1','.')
        look = list(filter(lambda x: x[0]=='2', inarr)) 
        look = list(map(lambda x: x[1], look))
        
        for i in look: tmp = tmp.replace('2', i,1)
        tmp = f'^{tmp}$'
        ls = list(filter(lambda x: bool(re.match(tmp, x)), ls))
        
    if '1' in pred: 
        tmp = pred.replace('0','.')
        tmp = tmp.replace('2','.')
        look = list(filter(lambda x: x[0]=='1', inarr))
        look = list(map(lambda x: x[1], look))
        for i in look: tmp = tmp.replace('1', i,1)
        tmp = f'^{tmp}$'
        ls = list(filter(lambda x: bool(re.match(tmp, x))==False, ls))
        
    return ls

def calcVal(word):
    ret = 0
    for i in word: ret+=freqs.loc[i].freq
    return ret

def validate(guess, word,g,verbose=True):
    fin=""
    val = ''
    for d,i in enumerate(guess): 
        if i in word and i==word[d]: 
            fin+='🟩'
            val+='2'
        elif i in word: 
            fin+= '🟨'
            val+='1'
        else: 
            fin+='⬛'
            val+='0'
    if verbose:
        print(fin, end = " | ")
        print(guess, end = " | ")
        print(len(g), end = " | ")
    return val
    
def getFilt(arr, drop=True, exts=True):
    posibs = arr
    if drop:
        reps = list(map(lambda x: len({i for i in x}), arr))
        minSiz = max(reps)
        posibs = list(filter(lambda x:len({i for i in x})==minSiz, arr))
    return posibs
    
def select(arr, drop=True, exts=True): 
    posibs = getFilt(arr, drop, exts)
    nu = list(map(calcVal, posibs))
    ser = list(zip(nu, posibs))
    if exts: maxScore = min(nu)
    else: maxScore = max(nu)
    l = nu.index(maxScore)
    
    return ser[l][1]

def run(i, srWord, verbose=True):
    turns=0
    setWord = wordList[i]
    g = wordList
    if verbose: print(f'set: {setWord}')
    pred = validate(srWord, setWord, g,verbose)

    turns+=1
    if verbose: print(turns)
    dro, ext = True, True
    while pred!= '22222' and turns!=5:
        turns+=1
        g = eliminate(g, srWord, pred)
        if turns==3: dro, ext = False, False
        srWord = select(g, dro, ext)
        pred = validate(srWord,setWord, g,verbose)
        if verbose: print(turns)
    return sum([int(i) for i in pred])


# savDict = []
# for j in guessArr:
#     print(j, end=" | ")
#     su = 0
#     for i in list(range(100)):
#         r = random.randint(0,2314)
#         su+=run(r, j)
#     savDict.append(su)
#     print(f'| {su}')

def avg(ret, div): return (sum(ret)/div)*100

# full=[]
# for j in list(range(20)):
#     su = []
#     for i in list(range(10)):
#             r = random.randint(0,2314)
#             su.append(run(r, 'abort', False))
#     su = list(filter(lambda x: x==10, su))
# #     print(su, end="")
#     print('=', end="")
#     full.append(avg(su, 100))
# print('>')
# print(avg(full, 2000))

wordList = wordList_orig
guessArr = guess(wordList)
for i in list(range(10)):
    r = random.randint(0,2314)
#     run(r, 'ratio')
    run(r, guessArr[0])

set: egret
⬛⬛🟨⬛⬛ | patio | 2315 | 1
⬛⬛⬛⬛⬛ | mucky | 356 | 2
🟨🟨🟨🟨⬛ | teeth | 87 | 3
🟨🟨⬛🟩🟩 | tenet | 66 | 4
🟨🟨⬛🟩🟩 | reset | 10 | 5
set: erect
⬛⬛🟨⬛⬛ | patio | 2314 | 1
⬛⬛🟨⬛⬛ | mucky | 356 | 2
🟨🟨🟩🟨⬛ | teeth | 109 | 3
⬛🟨🟩⬛🟨 | geese | 37 | 4
🟨⬛🟩🟨🟩 | tweet | 18 | 5
set: oaken
⬛🟩⬛⬛🟨 | patio | 2314 | 1
⬛🟩⬛🟨⬛ | gawky | 145 | 2
🟨🟩⬛🟩⬛ | easel | 71 | 3
⬛🟩⬛🟩⬛ | rarer | 9 | 4
🟩🟩🟩🟩🟩 | oaken | 2 | 5
set: strap
🟨🟨🟨⬛⬛ | patio | 2314 | 1
⬛⬛⬛🟨⬛ | jumpy | 1083 | 2
🟨⬛⬛🟨⬛ | teeth | 518 | 3
🟩🟩🟨🟨🟨 | start | 92 | 4
🟩🟩🟨⬛⬛ | stand | 9 | 5
set: rigid
⬛⬛⬛🟩⬛ | patio | 2314 | 1
⬛⬛⬛🟩⬛ | squib | 33 | 2
⬛⬛🟨🟩⬛ | eerie | 18 | 3
🟩🟩🟩🟩🟩 | rigid | 9 | 4
set: write
⬛⬛🟨🟨⬛ | patio | 2314 | 1
⬛⬛⬛⬛⬛ | mucky | 725 | 2
🟨🟨🟨🟨🟩 | eerie | 257 | 3
⬛🟨🟨⬛🟩 | geese | 60 | 4
🟨⬛🟨🟨🟩 | there | 29 | 5
set: award
⬛🟨⬛⬛⬛ | patio | 2314 | 1
⬛⬛⬛⬛⬛ | mucky | 498 | 2
⬛⬛⬛⬛⬛ | geese | 138 | 3
🟩🟩🟩🟩🟩 | award | 9 | 4
set: world
⬛⬛⬛⬛🟨 | patio | 2314 | 1
⬛⬛⬛⬛⬛ | mucky | 520 | 2
⬛⬛⬛⬛⬛ | geese | 145 | 3
⬛🟩⬛🟨🟨 | honor | 17 | 4
🟩🟩🟩🟩🟩 | world | 1 | 5
set: peril
🟩⬛⬛🟩

In [39]:
nudf = pd.DataFrame({'val': guessArr, 'score': savDict})
nudf = nudf.sort_values(by=['score'], ascending=False)
nudf.head()

val  score
6   atone    216
10  coast    214
18  ratio    200
20  talon    195
8   bloat    193

In [118]:
for i in list(range(10)):
    r = random.randint(0,2314)
    run(r, 'atone')

set: tribe
⬛🟨⬛⬛🟩 | atone | 2313 | 1
⬛⬛⬛⬛🟩 | fluke | 125 | 2
🟨🟨🟨🟨🟩 | eerie | 56 | 3
🟩🟨⬛🟨🟩 | tepee | 55 | 4
🟩⬛🟨⬛🟩 | these | 11 | 5
set: scout
⬛🟨🟩⬛⬛ | atone | 2312 | 1
⬛⬛🟩🟨⬛ | block | 111 | 2
🟨🟨🟩🟨⬛ | tooth | 57 | 3
⬛🟨🟩🟨🟩 | roost | 43 | 4
🟩🟨🟩🟩🟩 | stout | 3 | 5
set: lefty
⬛🟨⬛⬛🟨 | atone | 2312 | 1
⬛⬛⬛⬛🟩 | jumpy | 667 | 2
🟨🟩⬛🟩🟩 | testy | 52 | 3
⬛🟩⬛🟩🟩 | deity | 3 | 4
⬛🟩🟩🟩🟩 | hefty | 2 | 5
set: renal
🟨⬛⬛🟨🟨 | atone | 2312 | 1
⬛⬛⬛⬛⬛ | jumpy | 1156 | 2
🟨🟩🟨⬛🟨 | eerie | 491 | 3
⬛🟩🟨⬛🟨 | geese | 68 | 4
⬛🟩🟨⬛🟨 | heave | 46 | 5
set: rebel
⬛⬛⬛⬛🟨 | atone | 2312 | 1
⬛⬛⬛⬛⬛ | jumpy | 453 | 2
🟨🟩🟨⬛🟨 | eerie | 150 | 3
⬛🟩🟨⬛🟨 | geese | 27 | 4
🟨🟩🟨⬛⬛ | leech | 17 | 5
set: actor
🟩🟨🟨⬛⬛ | atone | 2312 | 1
🟩⬛⬛⬛⬛ | album | 60 | 2
🟩🟨🟨🟨🟨 | aorta | 21 | 3
🟩🟨🟩⬛🟨 | attic | 20 | 4
🟩🟩🟩🟩🟩 | actor | 2 | 5
set: thyme
⬛🟨⬛⬛🟩 | atone | 2312 | 1
⬛⬛⬛⬛🟩 | fluke | 125 | 2
🟨🟨⬛⬛🟩 | eerie | 56 | 3
🟩🟨⬛🟨🟩 | tepee | 8 | 4
🟩🟩🟨⬛🟩 | these | 3 | 5
set: putty
⬛🟨⬛⬛⬛ | atone | 2312 | 1
⬛🟩⬛🟨🟩 | jumpy | 283 | 2
⬛🟩⬛🟩🟩 | rusty | 37 | 3
🟩🟩🟩🟩🟩 | putty | 1 

In [51]:
wordList.index('overt')

1355

In [101]:
a = ['a','b']
# a.pop(a.index('a'))
# print(a)
a.index('c')

ValueError: 'c' is not in list